-What about electrons? (I don't see any - 2 phys.me)

-1e11 and mDM?

-Compute T(E,z_in, z_dep) by doing a delta function injection

# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [156]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_spectrum import nonrel_spec
from darkhistory.electrons.ics.ics_spectrum import rel_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.electrons import positronium as pos

from darkhistory.low_energy.lowE_deposition import compute_fs

# np.set_printoptions(threshold=np.nan)

In [4]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

## Photon Transfer Functions

In [7]:
user = 'hongwan'

highengphot_tflist_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [9]:
photeng = highengphot_tflist_arr[0].eng
eleceng = lowengelec_tflist_arr[0].eng

#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 60]
photeng_low  = photeng[photeng <= 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3000]
eleceng_low  = eleceng[eleceng <= 3000]


for highengphot_tflist in highengphot_tflist_arr:
    
#     highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
#         tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist._eng = photeng
    highengphot_tflist._in_eng = photeng
    highengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in highengphot_tflist._tflist])
    )
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
#     lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
#         tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist._eng = photeng
    lowengphot_tflist._in_eng = photeng
    lowengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengphot_tflist._tflist])
    )
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
#     lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
#         tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist._eng = eleceng
    lowengelec_tflist._in_eng = photeng
    lowengelec_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengelec_tflist._tflist])
    )
    
    for engloss in CMB_engloss_arr:
        engloss = np.pad(engloss, ((0,0),(photeng_low.size, 0)), 'constant')


In [10]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

highengphot_tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr)
lowengphot_tf_interp = tflist.TransferFuncInterp(xes, lowengphot_tflist_arr)
lowengelec_tf_interp = tflist.TransferFuncInterp(xes, lowengelec_tflist_arr)


## Inverse Compton Scattering

Set up all of the inverse Compton scattering transfer functions (needs to be hidden away at some point). 

In [107]:
Emax = 1e20
Emin = 1e-8
nEe = 500
nEp  = 500

dlnEp = np.log(Emax/Emin)/nEp
lowengEp_rel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

dlnEe = np.log(Emax/Emin)/nEe
lowengEe_rel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

Emax = 1e10
Emin = 1e-8
nEe = 500
nEp  = 500

dlnEp = np.log(Emax/Emin)/nEp
lowengEp_nonrel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)  

dlnEe = np.log(Emax/Emin)/nEe
lowengEe_nonrel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

print('********* Thomson regime scattered photon spectrum *********')
ics_thomson_ref_tf = nonrel_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(1000))
print('********* Relativistic regime scattered photon spectrum *********')
ics_rel_ref_tf = rel_spec(lowengEe_rel, lowengEp_rel, phys.TCMB(1000), inf_upp_bound=True)
print('********* Thomson regime energy loss spectrum *********')
engloss_ref_tf = engloss_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(1000), nonrel=True)

********* Thomson regime scattered photon spectrum *********
Initializing...
Computing spectra by an expansion in beta...
----> Computation by expansion in beta complete!
Computing spectra by analytic series...
*** Computing series 1/12...
*** Computing series 2/12...
*** Computing series 3/12...
*** Computing series 4/12...
*** Computing series 5/12...
*** Computing series 6/12...
*** Computing series 7/12...
*** Computing series 8/12...
*** Computing series 9/12...
*** Computing series 10/12...
*** Computing series 11/12...
*** Computing series 12/12...
----> Computation by analytic series complete!
Spectrum computed!
********* Relativistic regime scattered photon spectrum *********
Initializing...
Computing series 1/4...
Computing series 2/4...
Computing series 3/4...
Computing series 4/4...
Relativistic Computation Complete!
********* Thomson regime energy loss spectrum *********
Computing nonrelativistic energy loss spectrum...
Computing energy loss spectrum by beta expansion...
C

In [157]:
(ics_sec_phot_tf, ics_sec_elec_tf) = get_ics_cooling_tf(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
        eleceng, photeng, 1000
    )

***************************************************
injected energy:  256.3934702575207
low energy e:  241.77749941429153
scattered phot:  10929.660270052878
continuum_engloss:  10888.851956402641
diff:  40.808313650237324
deposited:  -26.19234280699629
***************************************************
***************************************************
injected energy:  271.8930080339778
low energy e:  241.77749941430983
scattered phot:  21860.040605517355
continuum_engloss:  21777.49965198327
diff:  82.54095353408411
deposited:  -52.425444914365436
***************************************************
***************************************************
injected energy:  288.3295262688189
low energy e:  241.77749941432006
scattered phot:  32797.66204970694
continuum_engloss:  32672.451652563712
diff:  125.21039714322615
deposited:  -78.65837028873169
***************************************************
***************************************************
injected energy:  305.7596674498

***************************************************
injected energy:  37639.91820934747
low energy e:  241.77749941692832
scattered phot:  969495.824703208
continuum_engloss:  929891.4842169137
diff:  39604.34048629436
deposited:  -2206.199776349816
***************************************************
***************************************************
injected energy:  39915.332374940335
low energy e:  241.77749941686815
scattered phot:  982272.9082651327
continuum_engloss:  940369.9915650168
diff:  41902.91670011589
deposited:  -2229.3618246033507
***************************************************
***************************************************
injected energy:  42328.30023010768
low energy e:  241.77749941687236
scattered phot:  995166.3025800211
continuum_engloss:  950827.258056268
diff:  44339.04452375311
deposited:  -2252.521793058482
***************************************************
***************************************************
injected energy:  44887.1371917972
low 

continuum_engloss:  1361601.5334371496
diff:  596874.2349061947
deposited:  -3194.054933531207
***************************************************
***************************************************
injected energy:  629825.8196346217
low energy e:  241.77749941824487
scattered phot:  2001130.6168929385
continuum_engloss:  1368337.67482428
diff:  632792.9420686585
deposited:  -3208.899932451955
***************************************************
***************************************************
injected energy:  667900.1476361318
low energy e:  241.77749941840528
scattered phot:  2045799.0198904495
continuum_engloss:  1374916.9758110775
diff:  670882.044079372
deposited:  -3223.6739422845562
***************************************************
***************************************************
injected energy:  708276.1508112764
low energy e:  241.77749941853995
scattered phot:  2092618.662518014
continuum_engloss:  1381345.8695863024
diff:  711272.7929317115
deposited:  -3238.419618

***************************************************
injected energy:  32145689.83390098
low energy e:  241.7774994183538
scattered phot:  33674085.51177932
continuum_engloss:  1525068.1132485673
diff:  32149017.398530755
deposited:  -2905.037272276769
***************************************************
***************************************************
injected energy:  34088966.05474694
low energy e:  241.777499418349
scattered phot:  35617361.73262265
continuum_engloss:  1525068.1132485373
diff:  34092293.61937412
deposited:  -2936.918441344406
***************************************************
***************************************************
injected energy:  36149717.51068711
low energy e:  241.77749941836743
scattered phot:  37678113.18856652
continuum_engloss:  1525068.1132486535
diff:  36153045.07531787
deposited:  -2976.543328393208
***************************************************
***************************************************
injected energy:  38335045.83282908
lo

***************************************************
injected energy:  680234814.24638
low energy e:  241.77749941842777
scattered phot:  681763209.9243128
continuum_engloss:  1525068.1132490349
diff:  680238141.8110638
deposited:  -3536.355987594067
***************************************************
***************************************************
injected energy:  721356474.5979489
low energy e:  241.77749941842822
scattered phot:  722884870.2758846
continuum_engloss:  1525068.1132490376
diff:  721359802.1626356
deposited:  -3538.367221282431
***************************************************
***************************************************
injected energy:  764964027.9303751
low energy e:  241.77749941842796
scattered phot:  766492423.6083105
continuum_engloss:  1525068.1132490363
diff:  764967355.4950614
deposited:  -3539.8695420259382
***************************************************
***************************************************
injected energy:  811207751.8311743
l

***************************************************
injected energy:  21708634824.262768
low energy e:  241.77749941842885
scattered phot:  21710163219.94075
continuum_engloss:  1525068.113249043
diff:  21708638151.8275
deposited:  -3566.981623881583
***************************************************
***************************************************
injected energy:  23020968579.083225
low energy e:  241.77749941842885
scattered phot:  23022496974.761215
continuum_engloss:  1525068.113249043
diff:  23020971906.647964
deposited:  -3567.0122971504293
***************************************************
***************************************************
injected energy:  24412635737.315914
low energy e:  241.7774994184288
scattered phot:  24414164132.993896
continuum_engloss:  1525068.1132490428
diff:  24412639064.880646
deposited:  -3567.044264001874
***************************************************
***************************************************
injected energy:  25888432174.15

***************************************************
injected energy:  342541522463.2824
low energy e:  241.77749941842885
scattered phot:  342653376535.9047
continuum_engloss:  1525068.1132490435
diff:  342651851467.7915
deposited:  -110329243.38176799
***************************************************
***************************************************
injected energy:  363248895635.07446
low energy e:  241.77749941842893
scattered phot:  363280303891.59644
continuum_engloss:  1525068.113249044
diff:  363278778823.4832
deposited:  -29883427.228246868
***************************************************
***************************************************
injected energy:  385208074137.1869
low energy e:  241.77749941842893
scattered phot:  385580529639.41565
continuum_engloss:  1525068.1132490437
diff:  385579004571.3024
deposited:  -370930672.8759045
***************************************************
***************************************************
injected energy:  408494732299.3

## Evolution

In [118]:
# mDM = 605662.1322589782*2
# mDM = 10**9.104*2
# mDM = 1.27164e9*2
mDM = 10**8*2
# mDM = 10**6.5*2

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mDM/2]), 
                                      photeng)
phot_spec_init.switch_spec_type()

elec_spec_init = spectools.rebin_N_arr(np.array([2]), 
    np.array([mDM/2 - phys.me]),
    photeng)

elec_spec_init.switch_spec_type()

In [119]:
print(phot_spec_init.toteng(bound_type = 'eng', bound_arr = np.array([1,1e13]))/mDM)

[1.]


In [142]:
%autoreload
phot_spec_init.rs = 3000
elec_spec_init.rs = 3000
end_rs   = 100
#mDM = 1e8
sigmav = 1e-26
tau = 1e25

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)


def evolve(
    in_spec_elec, in_spec_phot, 
    rate_func_N, rate_func_eng, xe_init, T_m_init,
    highengphot_tf_interp, lowengphot_tf_interp, 
    lowengelec_tf_interp, 
    end_rs, coarsen_factor=1, std_soln=False
):
    """
    Main function that computes the temperature and ionization history. 
    
    Parameters
    ----------
    in_spec_elec : Spectrum
        Spectrum per annihilation/decay into electrons. 
    in_spec_phot : Spectrum
        Spectrum per annihilation/decay into photons.
    rate_func_N : function
        Function describing the rate of annihilation/decay, dN/(dV dt)
    rate_func_eng : function
        Function describing the rate of annihilation/decay, dE/(dV dt)
    xe_init : float
        xe at the initial redshift. 
    T_m_init : float
        Matter temperature at the initial redshift. 
    highengphot_tf_interp : TransferFuncInterp
        Interpolation table for high energy photons. 
    lowengphot_tf_interp : TransferFuncInterp
        Interpolation table for low energy photons. 
    lowengelec_tf_interp : TransferFuncInterp
        Interpolation table for low energy electrons. 
    end_rs : float
        Final redshift to evolve to. 
    coarsen_factor : int
        Coarsening to apply to the transfer function matrix. 
    std_soln : bool
        If true, uses the standard TLA solution for f(z). 
    """
    
    # Initialize the next spectrum as None.
    next_highengphot_spec = None
    next_lowengphot_spec  = None
    next_lowengelec_spec  = None
    
    if (
        highengphot_tf_interp.dlnz    != lowengphot_tf_interp.dlnz
        or highengphot_tf_interp.dlnz != lowengelec_tf_interp.dlnz
        or lowengphot_tf_interp.dlnz  != lowengelec_tf_interp.dlnz
    ):
        raise TypeError('TransferFuncInterp objects must all have the same dlnz.')
    
    if in_spec_elec.rs != in_spec_phot.rs:
        raise TypeError('Input spectra must have the same rs.')
    
    # redshift/timestep related quantities. 
    dlnz = highengphot_tf_interp.dlnz
    prev_rs = None
    rs = in_spec_phot.rs
    dt = dlnz/phys.hubble(rs)
    
    # Function that changes the normalization from per annihilation to per baryon. 
    def norm_fac(rs): 
        return rate_func_N(rs) * dt / (phys.nB * rs**3)
    
    # ICS for in_spec_elec goes here.
    
    (ics_sec_phot_tf, ics_sec_elec_tf) = get_ics_cooling_tf(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
        eleceng, photeng, rs
    )
    
    ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2*norm_fac(rs)
    if positronium_phot_spec.spec_type != 'N':
        positronium_phot_spec.switch_spec_type()
        
    print('input electron energy: ', (phys.me*2 + in_spec_elec.toteng())*norm_fac(rs))
    print('ICS secondary photon spectrum energy: ', 
          ics_phot_spec.toteng(bound_type='eng', bound_arr=np.array([phys.rydberg, np.inf]))
         )
    print('ICS low energy electron spectrum energy: ', ics_lowengelec_spec.toteng() + phys.me*norm_fac(rs))
    print('Positronium spectrum energy: ', positronium_phot_spec.toteng())
    
    init_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec
    
    # Initialize the Spectra object that will contain all the 
    # output spectra during the evolution.
    out_highengphot_specs = Spectra([init_inj_spec], spec_type=init_inj_spec.spec_type)
    out_lowengphot_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    out_lowengelec_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    
    # Initialize the xe and T array that will store the solutions.
    xe_arr  = np.array([xe_init])
    T_m_arr = np.array([T_m_init])
    
    # Load the standard TLA solution if necessary. 
#     if std_soln:
    if True:
        soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
        xe_std  = interp1d(soln[0,:], soln[2,:])
        T_m_std = interp1d(soln[0,:], soln[1,:])

    # Define these methods for speed.
    append_highengphot_spec = out_highengphot_specs.append
    append_lowengphot_spec  = out_lowengphot_specs.append
    append_lowengelec_spec  = out_lowengelec_specs.append
    
    # Loop while we are still at a redshift above end_rs.
    while rs > end_rs:
        
        # If prev_rs exists, calculate xe and T_m. 
        if prev_rs is not None:
#             print('here at least?')
            
            # f_continuum, f_lyman, f_ionH, f_ionHe, f_heat
            # f_raw takes in dE/(dV dt)
            if std_soln:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
#                 print("\n SHOULD BE ONE: ", (next_lowengelec_spec.toteng() + next_lowengphot_spec.toteng())/(
#                 np.sum(f_raw) * rate_func_eng(rs) * dt / phys.nB / rs**3), '\n')   
            else:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
                
            std_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
            
            backreaction_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
            
            f_raw = np.float64(f_raw)
            
            init_cond = np.array([T_m_arr[-1], xe_arr[-1], 0, 0])

            new_vals = tla.get_history(
                init_cond, f_raw[2], f_raw[1], f_raw[4], 
                rate_func_eng, np.array([prev_rs, rs]),
                reion_switch = False
            )

            T_m_arr = np.append(T_m_arr, new_vals[-1,0])
            xe_arr  = np.append(xe_arr,  new_vals[-1,1])
            
        print('T_m at '+str(rs)+': '+ str(T_m_arr[-1]))
        print('x_e at '+str(rs)+': '+ str(xe_arr[-1]))
        print('Standard x_e at '+str(rs)+': '+str(xe_std(rs)))
        print('Standard T_m at '+str(rs)+': '+str(T_m_std(rs)))
        if prev_rs is not None:
            print('Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', backreaction_f)
            print('Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', std_f)
            
        if std_soln:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_std(rs))
        else:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_arr[-1])
        
#         if coarsen_factor > 1:
#             prop_tf = np.zeros_like(highengphot_tf._grid_vals)
#             for i in np.arange(coarsen_factor):
#                 prop_tf += matrix_power(highengphot_tf._grid_vals, i)
#             lowengphot_tf._grid_vals = np.matmul(prop_tf, lowengphot_tf._grid_vals)
#             lowengelec_tf._grid_vals = np.matmul(prof_tf, lowengelec_tf._grid_vals)
#             highengphot_tf._grid_vals = matrix_power(
#                 highengphot_tf._grid_vals, coarsen_factor
#             )
            
        next_highengphot_spec = highengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengphot_spec  = lowengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1]) + ics_lowengelec_spec
        
#         print('\n SHOULD ALSO BE ONE: ', 
#               out_highengphot_specs[-1].toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))/(
#         next_highengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengelec_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))), '\n')
        
#         print('ENERGIES:')
#         print(out_highengphot_specs[-1].toteng())
#         for i in np.arange(500):
#             print(i, highengphot_tf.in_eng[i], highengphot_tf[i].toteng())
    
#         print('\n SHOULD ALSO BE ONE (JUST HIGH ENERGY): ', 
#               out_highengphot_specs[-1].toteng()/(
#         next_highengphot_spec.toteng()), '\n')
        
#         output_eng = next_highengphot_spec.N*next_highengphot_spec.eng/norm_fac(rs)
#         output_N   = next_highengphot_spec.dNdE/norm_fac(rs)
#         utils.compare_arr([next_highengphot_spec.eng, output_eng])
#         print(output_eng[385:391]/1e9)
#         print(output_N[385:391])
#         print(next_highengphot_spec.eng[385:391])
        
#         utils.compare_arr([out_highengphot_specs[-1].N, next_highengphot_spec.N])
        
        # Re-define existing variables.
        prev_rs = rs
        rs = np.exp(np.log(rs) - 0.002)
        
        dt = dlnz/phys.hubble(rs)
        next_highengphot_spec.rs = rs
        next_lowengphot_spec.rs  = rs
        next_lowengelec_spec.rs  = rs
        
        # Add the next injection spectrum to next_highengphot_spec
    
        (ics_sec_phot_tf, ics_sec_elec_tf) = get_ics_cooling_tf(
            ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
            eleceng, photeng, rs
        )

        ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2 * norm_fac(rs)
        if positronium_phot_spec.spec_type != 'N':
            positronium_phot_spec.switch_spec_type()

        next_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec

        # This keeps the redshift. 
        next_highengphot_spec.N += next_inj_spec.N
        
        append_highengphot_spec(next_highengphot_spec)
        append_lowengphot_spec(next_lowengphot_spec)
        append_lowengelec_spec(next_lowengelec_spec)
        
        print('************************ END OF LOOP **************************')
        
        
    return (
        xe_arr, T_m_arr, 
        out_highengphot_specs, out_lowengphot_specs, out_lowengelec_specs
    )

In [143]:
%%prun
# a = evolve(
#         phot_spec_init*0, phot_spec_init, 
#         rate_func_decay_N, rate_func_decay_eng, .9999, phys.TCMB(phot_spec_init.rs),
#         highengphot_tf_interp, lowengphot_tf_interp, 
#         lowengelec_tf_interp, 
#         end_rs, coarsen_factor=1, std_soln=False
#     )

b_t = evolve(
        elec_spec_init, phot_spec_init*0, 
        rate_func_decay_N, rate_func_decay_eng, xe_std(phot_spec_init.rs), phys.TCMB(phot_spec_init.rs),
        #rate_func_decay_N, rate_func_decay_eng, 2e-4, phys.TCMB(phot_spec_init.rs),
        highengphot_tf_interp, lowengphot_tf_interp, 
        lowengelec_tf_interp, 
        end_rs, coarsen_factor=1, std_soln=True
    )

input electron energy:  3.7837473762910873e-06
ICS secondary photon spectrum energy:  [5.12428574e-05]
ICS low energy electron spectrum energy:  9.676602857757566e-09
Positronium spectrum energy:  1.9334909215821445e-08
T_m at 3000: 0.70459598745
x_e at 3000: 0.999999999999
Standard x_e at 3000: 0.999999999999
Standard T_m at 3000: 0.70459598745
************************ END OF LOOP **************************
Normalized electron heat, lyman, ionH, ionHe, cont:  0.9999999999999963 1.0000061761358293e-15 1.000006438199948e-15 1.0000070572768831e-15 1.0000059641221025e-15
Goods:  0.3641110635027005
Split by photon and electron deposition!
f_phot:  [1.02774605 0.03372262 0.07793894 0.         0.        ]
f_elec:  [3.64112869e-16 3.64112936e-16 3.64112659e-16 3.64113546e-16
 3.64111064e-01]
Normalized electron heat, lyman, ionH, ionHe, cont:  0.9999999999999963 1.0000061761358293e-15 1.000006438199948e-15 1.0000070572768831e-15 1.0000059641221025e-15
Goods:  0.3641110635027005
Split by photo

../darkhistory/spec/spectrum.py:748: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)


************************ END OF LOOP **************************
Normalized electron heat, lyman, ionH, ionHe, cont:  0.9999999999999963 1.0000064008310856e-15 1.0000066724294461e-15 1.0000073140291975e-15 1.0000061811040177e-15
Goods:  0.7432880184612947
Split by photon and electron deposition!
f_phot:  [2.67891107 0.09205403 0.20298104 0.         0.        ]
f_elec:  [7.43291439e-16 7.43291566e-16 7.43291180e-16 7.43293003e-16
 7.43288018e-01]
Normalized electron heat, lyman, ionH, ionHe, cont:  0.9999999999999963 1.0000064008310856e-15 1.0000066724294461e-15 1.0000073140291975e-15 1.0000061811040177e-15
Goods:  0.7432880184612947
Split by photon and electron deposition!
f_phot:  [2.67891107 0.09205403 0.20298104 0.         0.        ]
f_elec:  [7.43291439e-16 7.43291566e-16 7.43291180e-16 7.43293003e-16
 7.43288018e-01]
Normalized electron heat, lyman, ionH, ionHe, cont:  0.9999999999999963 1.000006033998064e-15 1.0000062900310136e-15 1.0000068948604647e-15 1.000005826863641e-15
Good

KeyboardInterrupt: 

In [15]:
plt.figure()
# plt.plot(a[3].rs, a[0])
plt.plot(b[3].rs, b[0])
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


plt.figure()
# plt.plot(a[3].rs, a[1]/phys.kB)
plt.plot(b[3].rs, b[1]/phys.kB)
plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


NameError: name 'b' is not defined

<Figure size 720x720 with 0 Axes>

In [ ]:
test_tf = highengphot_tf_interp.get_tf(4, xe_std(3000))
highengphot_tf_interp.xe

In [ ]:
plt.figure()

ax = plt.gca()

print(test_tf.in_eng[375])

test_tf.plot(ax, ind=250, indtype='ind')
test_tf.plot(ax, ind=377, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
test_spectra = highengphot_tf_interp._grid_vals[-1,-1,:,:]

plt.figure()
ax = plt.gca()

plt.plot(highengphot_tf_interp.eng, test_spectra[370, :])

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
tf_before_interp = highengphot_tflist_arr[-1][-1]
tf_before_interp_2 = highengphot_tflist_arr[-1][0]

# print(tf_before_interp.in_eng[376])

plt.figure()
ax = plt.gca()
# tf_before_interp.plot(ax, ind=200, indtype='ind')
tf_before_interp.plot(ax, ind=370, indtype='ind')
# tf_before_interp_2.plot(ax, ind=376, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
lowengphot_spectra = b[3]
print(lowengphot_spectra.in_eng)

lowengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengphot_spectra[0].toteng())

plt.figure()
ax = plt.gca()

lowengphot_spectra.switch_spec_type()
lowengphot_spectra.plot(ax, ind=0, indtype='ind')
lowengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4, 1e3, 1e-10, 1e6])

In [ ]:
lowengelec_spectra = b[4]

lowengelec_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengelec_spectra[0].toteng())
utils.compare_arr([lowengelec_spectra[0].eng, lowengelec_spectra[0].dNdE])

plt.figure()
ax = plt.gca()

lowengelec_spectra.switch_spec_type()
lowengelec_spectra.plot(ax, ind=0, indtype='ind')
lowengelec_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1, 3e3, 1e-4, 1e3])

In [ ]:
highengphot_spectra = b[2]

highengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(2*10**9.104)
print(highengphot_spectra[0].toteng())
print(highengphot_spectra[1].toteng())

plt.figure()
ax = plt.gca()

highengphot_spectra.switch_spec_type()
highengphot_spectra.plot(ax, ind=1, indtype='ind')
highengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4,5e12, 1e-40, 1e10])